In [15]:
from pathlib import Path
from omegaconf import OmegaConf


dir = Path("/workspace/src/output/results_old/walker2d_uni")

# # Create directory ablation_1/ if it does not exist
# ablation_dir = dir / "ablation_1"
# ablation_dir.mkdir(exist_ok=True)

# # Create directory ablation_2/ if it does not exist
# ablation_dir = dir / "ablation_2"
# ablation_dir.mkdir(exist_ok=True)

# # Create directory ablation_3/ if it does not exist
# ablation_dir = dir / "ablation_3"
# ablation_dir.mkdir(exist_ok=True)

dir_iter = list(dir.iterdir())
for algo_dir in dir_iter:
	if not algo_dir.is_dir() or algo_dir.name != "dcg_me":
		continue
	algo_dir_iter = list(algo_dir.iterdir())
	for run_dir in algo_dir_iter:
		if not run_dir.is_dir():
			continue
		config = OmegaConf.load(run_dir / ".hydra" / "config.yaml")
		(run_dir / ".hydra" / "config.yaml").rename(run_dir / ".hydra" / "config.yaml.old")
		if "dc_actor" not in config.algo:
			config.algo.dc_actor = True
		if config.algo.name == "dcg_me" and config.algo.dc_actor == False and config.algo.ai_batch_size == 0:
			# Ablation Actor
			config.algo.name = "ablation_1"
			OmegaConf.save(config=config, f=run_dir / ".hydra" / "config.yaml")
			ablation_dir = dir / "ablation_1"
			ablation_dir.mkdir(exist_ok=True)
			run_dir.rename(ablation_dir / run_dir.name)
		elif config.algo.name == "dcg_me" and config.algo.actor_batch_size == 0 and config.algo.qpg_batch_size == 128 and config.algo.ai_batch_size == 0:
			# Ablation AI
			config.algo.name = "ablation_2"
			OmegaConf.save(config=config, f=run_dir / ".hydra" / "config.yaml")
			ablation_dir = dir / "ablation_2"
			ablation_dir.mkdir(exist_ok=True)
			run_dir.rename(ablation_dir / run_dir.name)
		elif config.algo.name == "dcg_me" and config.algo.actor_batch_size == 64 and config.algo.qpg_batch_size == 128 and config.algo.ai_batch_size == 0:
			# DCG-MAP-Elites
			config.algo.name = "ablation_3"
			OmegaConf.save(config=config, f=run_dir / ".hydra" / "config.yaml")
			ablation_dir = dir / "ablation_3"
			ablation_dir.mkdir(exist_ok=True)
			run_dir.rename(ablation_dir / run_dir.name)
		elif config.algo.name == "dcg_me" and config.algo.dc_actor == True and config.algo.actor_batch_size == 0 and config.algo.qpg_batch_size == 64 and config.algo.ai_batch_size == 64:
			# DCRL-MAP-Elites
			OmegaConf.save(config=config, f=run_dir / ".hydra" / "config.yaml")
		else:
			raise ValueError

# Visualization

In [1]:
import functools
from pathlib import Path

import jax
import jax.numpy as jnp

from qdax.environments import behavior_descriptor_extractor
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.core.neuroevolution.networks.networks import MLP
from qdax.tasks.brax_envs import reset_based_scoring_function_brax_envs as scoring_function

from utils import get_env, get_config, get_repertoire


run_dir = Path("/project/output/results/humanoid_uni/dcg_me/2023-10-12_070304_680643")
config = get_config(run_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/project/output/results/humanoid_uni/dcg_me/2023-10-12_070304_680643/.hydra/config.yaml'

In [2]:
repertoire = get_repertoire(run_dir)

In [3]:
# Init a random key
random_key = jax.random.PRNGKey(config.seed)

# Init environment
env = get_env(config)
reset_fn = jax.jit(env.reset)

# Init policy network
policy_layer_sizes = config.policy_hidden_layer_sizes + (env.action_size,)
policy_network = MLP(
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.lecun_uniform(),
    final_activation=jnp.tanh,
)

# Define the fonction to play a step with the policy in the environment
def play_step_fn(env_state, policy_params, random_key):
    actions = policy_network.apply(policy_params, env_state.obs)
    state_desc = env_state.info["state_descriptor"]
    next_state = env.step(env_state, actions)

    transition = QDTransition(
        obs=env_state.obs,
        next_obs=next_state.obs,
        rewards=next_state.reward,
        dones=next_state.done,
        truncations=next_state.info["truncation"],
        actions=actions,
        state_desc=state_desc,
        next_state_desc=next_state.info["state_descriptor"],
        desc=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
        desc_prime=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
    )

    return next_state, policy_params, random_key, transition

# Prepare the scoring function
bd_extraction_fn = behavior_descriptor_extractor[config.env.name]
scoring_fn = functools.partial(
    scoring_function,
    episode_length=config.env.episode_length,
    play_reset_fn=reset_fn,
    play_step_fn=play_step_fn,
    behavior_descriptor_extractor=bd_extraction_fn,
)

In [4]:
NUM_EVALUATION = 4
random_keys = jax.random.split(random_key, NUM_EVALUATION)
fitnesses, descriptors, extra_scores, random_keys = jax.vmap(scoring_fn, in_axes=(None, 0))(repertoire.genotypes, random_keys)

## Import

In [14]:
import functools
import pickle

import jax
import jax.numpy as jnp
from flax import serialization

from qdax import environments
from qdax.tasks.brax_envs import reset_based_scoring_function_brax_envs as scoring_function
from qdax.tasks.brax_envs import reset_based_scoring_actor_dc_function_brax_envs as scoring_actor_dc_function
from qdax.core.neuroevolution.buffers.buffer import QDTransition
from qdax.core.neuroevolution.networks.networks import MLP, MLPDC

from brax.v1.io import html
from IPython.display import HTML
from omegaconf import OmegaConf
from utils import get_env, get_config

## Load experiment

In [15]:
from pathlib import Path


run_dir = Path("/project/output/results/ant_omni/dcg_me/2023-10-11_075315_268825")
config = get_config(run_dir)

In [3]:
# Init a random key
random_key = jax.random.PRNGKey(config.seed)

# Init environment
env = get_env(config)
reset_fn = jax.jit(env.reset)
step_fn = jax.jit(env.step)

# Init policy network
policy_layer_sizes = config.policy_hidden_layer_sizes + (env.action_size,)
policy_network = MLP(
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.lecun_uniform(),
    final_activation=jnp.tanh,
)
actor_dc_network = MLPDC(
    layer_sizes=policy_layer_sizes,
    kernel_init=jax.nn.initializers.lecun_uniform(),
    final_activation=jnp.tanh,
)

def play_step_fn(env_state, policy_params, random_key):
    actions = policy_network.apply(policy_params, env_state.obs)
    state_desc = env_state.info["state_descriptor"]
    next_state = env.step(env_state, actions)

    transition = QDTransition(
        obs=env_state.obs,
        next_obs=next_state.obs,
        rewards=next_state.reward,
        dones=next_state.done,
        truncations=next_state.info["truncation"],
        actions=actions,
        state_desc=state_desc,
        next_state_desc=next_state.info["state_descriptor"],
        desc=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
        desc_prime=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
    )

    return next_state, policy_params, random_key, transition

# Prepare the scoring function
bd_extraction_fn = environments.behavior_descriptor_extractor[config.env.name]
scoring_fn = functools.partial(
    scoring_function,
    episode_length=config.env.episode_length,
    play_reset_fn=reset_fn,
    play_step_fn=play_step_fn,
    behavior_descriptor_extractor=bd_extraction_fn,
)

def play_step_actor_dc_fn(env_state, actor_dc_params, desc, random_key):
    actions = actor_dc_network.apply(actor_dc_params, env_state.obs, desc/config.env.max_bd)
    state_desc = env_state.info["state_descriptor"]
    next_state = env.step(env_state, actions)

    transition = QDTransition(
        obs=env_state.obs,
        next_obs=next_state.obs,
        rewards=next_state.reward,
        dones=next_state.done,
        truncations=next_state.info["truncation"],
        actions=actions,
        state_desc=state_desc,
        next_state_desc=next_state.info["state_descriptor"],
        desc=jnp.zeros(env.behavior_descriptor_length,) * jnp.nan,
        desc_prime=desc/config.env.max_bd,
    )

    return next_state, actor_dc_params, desc, random_key, transition

# Prepare the scoring function
scoring_actor_dc_fn = jax.jit(functools.partial(
    scoring_actor_dc_function,
    episode_length=config.env.episode_length,
    play_reset_fn=reset_fn,
    play_step_actor_dc_fn=play_step_actor_dc_fn,
    behavior_descriptor_extractor=bd_extraction_fn,
))

# Build the reconstruction function
fake_obs = jnp.zeros(shape=(env.observation_size,))
fake_desc = jnp.zeros(shape=(env.behavior_descriptor_length,))

## Load repertoire

In [4]:
from jax.flatten_util import ravel_pytree
from qdax.core.containers.mapelites_repertoire import MapElitesRepertoire


random_key, random_subkey = jax.random.split(random_key)
fake_params = policy_network.init(random_subkey, fake_obs)

_, reconstruction_fn = ravel_pytree(fake_params)

# Build the repertoire
repertoire = MapElitesRepertoire.load(reconstruction_fn=reconstruction_fn, path=str(run_dir) + "/repertoire/")

## Rollout

### Policy

In [15]:
descriptor_desired = jnp.array([30, 30])
index = jnp.argmin(jnp.linalg.norm(repertoire.descriptors - descriptor_desired, axis=1))
descriptor_repertoire = repertoire.descriptors[index]
policy_params = jax.tree_util.tree_map(lambda x: x[index], repertoire.genotypes)

random_key, subkey = jax.random.split(random_key)
state = reset_fn(subkey)
rollout = [state]
while True:
    actions = policy_network.apply(policy_params, state.obs)
    state = step_fn(state, actions)

    if state.done:
        break
    else:
        rollout.append(state)

if env.state_descriptor_name == "feet_contact":
    descriptor = sum([s.info["state_descriptor"] for s in rollout]) / len(rollout)
elif env.state_descriptor_name == "xy_position":
    descriptor = rollout[-1].info["state_descriptor"]
else:
    raise NotImplementedError

print("Episode length: {}/{}".format(len(rollout), config.env.episode_length))
print("Fitness: {}".format(sum([s.reward for s in rollout])))
print("Desired descriptor: {}".format(descriptor_desired))
print("Repertoire descriptor: {}".format(descriptor_repertoire))
print("Observed descriptor: {}".format(descriptor))

Episode length: 250/250
Fitness: 849.4724731445312
Desired descriptor: [30 30]
Repertoire descriptor: [28.760014 29.178423]
Observed descriptor: [29.251493 27.96182 ]


In [16]:
a = html.render(env.sys, [state.qp for state in rollout])
with open(run_dir / "rollout.html", "w") as f:
    f.write(a)

HTML(a)

### Descriptor-conditioned actor

In [17]:
random_key, subkey = jax.random.split(random_key)
actor_dc_params = actor_dc_network.init(subkey, fake_obs, fake_desc)

with open(run_dir / "actor.pickle", "rb") as params_file:
    state_dict = pickle.load(params_file)
actor_dc_params = serialization.from_state_dict(actor_dc_params, state_dict)

In [18]:
def normalize_desc(desc):
    return 2*(desc - env.behavior_descriptor_limits[0])/(env.behavior_descriptor_limits[1] - env.behavior_descriptor_limits[0]) - 1

In [21]:
descriptor_desired = jnp.array([30, 30])

random_key, subkey = jax.random.split(random_key)
state = reset_fn(subkey)
rollout = [state]
while True:
    descriptor_desired_normalized = normalize_desc(descriptor_desired)
    actions = actor_dc_network.apply(actor_dc_params, state.obs, descriptor_desired_normalized)
    state = step_fn(state, actions)

    if state.done:
        break
    else:
        rollout.append(state)

if env.state_descriptor_name == "feet_contact":
    descriptor = sum([s.info["state_descriptor"] for s in rollout]) / len(rollout)
elif env.state_descriptor_name == "xy_position":
    descriptor = rollout[-1].info["state_descriptor"]
else:
    raise NotImplementedError

print("Episode length: {}/{}".format(len(rollout), config.env.episode_length))
print("Fitness: {}".format(sum([s.reward for s in rollout])))
print("Desired descriptor: {}".format(descriptor_desired))
print("Observed descriptor: {}".format(descriptor))

Episode length: 250/250
Fitness: 845.4260864257812
Desired descriptor: [30 30]
Observed descriptor: [27.339926 26.786737]


In [22]:
a = html.render(env.sys, [state.qp for state in rollout])
with open(run_dir / "rollout.html", "w") as f:
    f.write(a)

HTML(a)